In [1]:
%matplotlib inline

import matplotlib.pylab as plt
import pandas as pd
import numpy as np

from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
df1 = pd.read_csv('Data1.csv')
#df2 = pd.read_csv('Data2.csv')
#df3 = pd.read_csv('Data3.csv')
#df4 = pd.read_csv('Data4.csv')

y1 = pd.Series(df1['id'], index=df1.id)


In [5]:
df1

,time,speed,ax,ay,az,at,id
0,12:00:00 AM,750.9325,-0.256311,0.182557,0.237720,0.379743,1
1,12:01:07 AM,709.4121,0.028039,0.115812,0.071011,0.033220,1
2,12:02:25 AM,713.0659,0.381876,0.151775,0.029867,0.033829,1
3,12:03:43 AM,1025.1830,0.315436,0.490983,0.780819,0.792705,1
4,12:05:02 AM,1027.8420,-1.172448,-0.411439,0.735713,0.466907,1
...,...,...,...,...,...,...,...
2977,8:05:09 PM,880.6443,0.807029,1.102046,0.586986,-0.015238,596
2978,8:07:07 PM,1111.1610,0.309036,-0.029258,-0.092650,0.522375,596
2979,8:11:43 PM,803.4224,-0.742418,-0.474221,-0.120689,-0.015848,596
2980,8:13:02 PM,918.9009,0.537004,-0.386447,-0.455935,-0.153604,597


In [6]:
y1

id
1        1
1        1
1        1
1        1
1        1
      ... 
596    120
596    120
596    120
597    120
597    120
Name: id, Length: 2982, dtype: int64

In [8]:
df1.head(50)

,time,speed,ax,ay,az,at,id
0,12:00:00 AM,750.9325,-0.256311,0.182557,0.237720,0.379743,1
1,12:01:07 AM,709.4121,0.028039,0.115812,0.071011,0.033220,1
2,12:02:25 AM,713.0659,0.381876,0.151775,0.029867,0.033829,1
3,12:03:43 AM,1025.1830,0.315436,0.490983,0.780819,0.792705,1
4,12:05:02 AM,1027.8420,-1.172448,-0.411439,0.735713,0.466907,1
5,12:06:51 AM,878.0190,-0.167318,-0.404734,0.206024,0.030172,2
6,12:07:40 AM,1027.1500,1.911513,1.257173,1.638745,1.168486,2
7,12:08:59 AM,1034.8610,1.397367,1.097779,0.402905,0.368466,2
8,12:10:21 AM,892.9366,-0.746075,-0.629958,-0.547061,-0.366637,2
9,12:11:36 AM,1029.2110,0.674150,0.928327,0.155432,-0.850306,2


In [10]:
extraction_settings = ComprehensiveFCParameters()

X = extract_features(df1, column_id='id', 
                     column_sort='time',
                     default_fc_parameters=extraction_settings,
                     # we impute = remove all NaN features automatically
                     impute_function=impute)


Feature Extraction: 100%|██████████| 10/10 [01:19<00:00,  7.94s/it]


In [11]:
X.to_csv(r'C:\Users\USER\Documents\GitHub\CBM_ML_Test\Test_Windmill_Extracted_Label_1.0.csv')

In [12]:
X_extracted = X
X_extracted

,at__variance_larger_than_standard_deviation,at__has_duplicate_max,at__has_duplicate_min,at__has_duplicate,at__sum_values,at__abs_energy,at__mean_abs_change,at__mean_change,at__mean_second_derivative_central,at__median,...,az__fourier_entropy__bins_2,az__fourier_entropy__bins_3,az__fourier_entropy__bins_5,az__fourier_entropy__bins_10,az__fourier_entropy__bins_100,az__permutation_entropy__dimension_3__tau_1,az__permutation_entropy__dimension_4__tau_1,az__permutation_entropy__dimension_5__tau_1,az__permutation_entropy__dimension_6__tau_1,az__permutation_entropy__dimension_7__tau_1
1,0.0,0.0,0.0,0.0,1.706403,0.992835,0.357952,0.021791,0.003454,0.379743,...,0.636514,0.636514,1.098612,1.098612,1.098612,1.098612,0.693147,-0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.350180,2.359480,0.789276,-0.220120,-0.270330,0.030172,...,0.636514,0.636514,1.098612,1.098612,1.098612,1.098612,0.693147,-0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,1.480265,7.257579,1.979096,0.451287,-0.642098,0.449230,...,0.636514,1.098612,1.098612,1.098612,1.098612,0.636514,0.693147,-0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.765885,12.875813,3.288690,-0.065906,0.859094,0.767714,...,0.636514,1.098612,1.098612,1.098612,1.098612,1.098612,0.693147,-0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,3.862646,30.188748,2.956872,0.500431,-1.501395,1.398282,...,0.636514,0.636514,1.098612,1.098612,1.098612,0.636514,0.693147,-0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,0.0,0.0,0.0,0.0,-1.057550,1.099630,0.434753,-0.126327,-0.062071,-0.207243,...,0.636514,0.636514,0.636514,1.098612,1.098612,1.098612,0.693147,-0.0,0.0,0.0
594,0.0,0.0,0.0,0.0,0.934423,0.652760,0.500736,0.119470,0.075888,0.094478,...,0.636514,0.636514,0.636514,1.098612,1.098612,0.636514,0.693147,-0.0,0.0,0.0
595,1.0,0.0,0.0,0.0,-4.057699,13.864617,1.027835,0.931527,0.185300,-0.182252,...,0.636514,0.636514,1.098612,1.098612,1.098612,0.636514,0.693147,-0.0,0.0,0.0
596,0.0,0.0,0.0,0.0,0.340732,0.423689,0.481688,-0.051049,-0.001829,-0.015238,...,0.636514,0.636514,0.636514,0.636514,1.098612,1.098612,0.693147,-0.0,0.0,0.0


In [14]:
y1

id
1        1
1        1
1        1
1        1
1        1
      ... 
596    120
596    120
596    120
597    120
597    120
Name: id, Length: 2982, dtype: int64

In [13]:
X_selected = select_features(X_extracted,y1)

AssertionError: X and y must contain the same number of samples.